In [56]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

## The panelists by country

Reading in the panel information Netquest gave us.

In [59]:
panels = pd.read_csv("../Data/AR/panel/190912/VANDUS 1147438_W3_20190912.csv",
                     sep=';', 
#                      skiprows=2891450,
#                      nrows=1000000,
                     na_values=[' ','.'], 
                     encoding='latin1',
                     #error_bad_lines=False
                    )

C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (185) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [60]:
panels.columns

Index(['p_codigo', 'p_sexo', 'panelistAge', 'pv_fecha_modificacion', 'pais',
       'BR_estado', 'BR_cidade', 'CO_departamento', 'CO_municipio',
       'MX_estado',
       ...
       'P12', 'P13', 'CLCSOCIAL2012', 'CLCSOCIAL2015', 'CLCSOCIAL2018',
       'MXCSOCIAL2018', 'AR_provincia', 'AR_localidad', 'Códigopostal',
       'AR_departamento'],
      dtype='object', length=187)

Creating a dictionary of individual country dataframes, and cleaning them up

In [61]:
panels_dict = {}
for p in panels.pais.unique():
    panels_dict[p] = panels[panels.pais==p]#.dropna(axis=1,how='all')

In [62]:
panels_dict.keys()

dict_keys(['AR'])

In [63]:
for pais, data in panels_dict.items():
    if not os.path.isdir("../Data/"+pais):
        os.mkdir("../Data/"+pais)
    data.to_csv("../Data/"+pais+"/"+pais+"_netquest-panel.csv", encoding='utf8')

Next, bring the variables and labels in order.

In [73]:
variables = pd.read_excel("../Data/AR/panel/190912/manual_variables.xlsx")
levels = pd.read_excel("../Data/AR/panel/190912/manual_levels.xlsx")

In [74]:
variables.Variable
levels.Variable
#[c for c in panels_dict["AR"].columns if not c in variables.Variable]

# [c for c in variables.Variable if not c in panels_dict["AR"].columns]
# variables.Variable

0                 p_sexo
1                 p_sexo
2              BR_estado
3              BR_estado
4              BR_estado
5              BR_estado
6              BR_estado
7              BR_estado
8              BR_estado
9              BR_estado
10             BR_estado
11             BR_estado
12             BR_estado
13             BR_estado
14             BR_estado
15             BR_estado
16             BR_estado
17             BR_estado
18             BR_estado
19             BR_estado
20             BR_estado
21             BR_estado
22             BR_estado
23             BR_estado
24             BR_estado
25             BR_estado
26             BR_estado
27             BR_estado
28             BR_estado
29             BR_cidade
              ...       
16535    AR_departamento
16536    AR_departamento
16537    AR_departamento
16538    AR_departamento
16539    AR_departamento
16540    AR_departamento
16541    AR_departamento
16542    AR_departamento
16543    AR_departamento


In [75]:
variables_dict = {}
levels_dict = {}

for pais in panels_dict:
    other_countries = [p for p in panels_dict if not p==pais]
    var_df = variables[(variables.Variable.str.startswith(pais)) #country-specific var
                       | (variables.Variable.apply(lambda s: sum([k in s for k in other_countries]))==0)] #not specific for another country
    lev_df = levels[levels.Variable.isin(var_df.Variable)]
    
    variables_dict[pais] = var_df
    levels_dict[pais] = lev_df
    
    var_df.to_excel('../Data/'+pais+'/'+pais+'_variables.xlsx')
    lev_df.to_excel('../Data/'+pais+'/'+pais+'_levels.xlsx')

In [76]:
lev_df

,Variable,Valor,Etiqueta
0,p_sexo,1,Hombre
1,p_sexo,2,Mujer
2,BR_estado,2816,Acre
3,BR_estado,2817,Alagoas
4,BR_estado,2818,Amazonas
5,BR_estado,2819,Amapá
6,BR_estado,2820,Bahia
7,BR_estado,2821,Ceará
8,BR_estado,2822,Distrito Federal
9,BR_estado,2823,Espírito Santo


In [ ]:
MX = pd.read_csv('../Data/MX/MX_netquest-panel.csv')

In [ ]:
print(sorted(MX.MX_municipio_name1.unique().astype(str)))